In [ ]:
!nvidia-smi

Wed Nov 16 12:56:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -q torchmetrics
!pip install -q transformers==4.20.1
!pip install -q sentencepiece
!pip install -q kaggle --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |████████████████▎               | 834.1 MB 1.2 MB/s eta 0:10:46tcmalloc: large alloc 1147494400 bytes == 0x3a528000 @  0x7f8953856615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████▋           | 1055.7 MB 1.2 MB/s eta 0:07:49tcmalloc: large alloc 1434370048 bytes == 0x7eb7e000 @  0x7f8953856615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |██████████████████████████▏     | 

In [ ]:
prepare_data = True
import os
if os.path.exists('/content/data'):
    prepare_data = False

In [ ]:
if prepare_data:    
    from google.colab import files, drive
    
    # Upload your kaggle.json (kaggle api)
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
    
    # Then move kaggle.json into the folder where the API expects to find it.
    !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

    !mkdir data && cd data && kaggle competitions download -c feedback-prize-english-language-learning
    !unzip /content/data/feedback-prize-english-language-learning.zip -d /content/data/

    drive.mount('/content/drive')

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes
100% 2.80M/2.80M [00:01<00:00, 2.21MB/s]
100% 2.80M/2.80M [00:01<00:00, 1.71MB/s]
Archive:  /content/data/feedback-prize-english-language-learning.zip
  inflating: /content/data/sample_submission.csv  
  inflating: /content/data/test.csv  
  inflating: /content/data/train.csv  
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import time
import datetime
import pytz
import gc; gc.enable()
import random
import warnings
import yaml
from itertools import chain
from pathlib import Path
from tabulate import tabulate
warnings.filterwarnings("ignore")
print(f'python version: {sys.version}') 

os.system('pip install -q iterative-stratification==0.1.7')
import iterstrat
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
print(f'iterstart version: {iterstrat.__version__}')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim.swa_utils import AveragedModel, SWALR

print(f'torch version: {torch.__version__}')

from torchmetrics.functional import mean_squared_error

# os.system('pip install --root-user-action=ignore --force-reinstall transformers==4.22.1')
import transformers

from transformers import AutoTokenizer, AutoModel, AutoConfig, DataCollatorWithPadding
print(f'transfromers version: {transformers.__version__}')

os.environ["TOKENIZERS_PARALLELISM"] = "false"

python version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0]
iterstart version: 0.1.6
torch version: 1.11.0+cu113
transfromers version: 4.20.1


# Config

In [ ]:
class BASICCONF:
    seed = 12
    
    data_path = '/content/data'
    
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    num_labels = 6
    num_folds = 4

# Seeding

In [ ]:
#https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(BASICCONF.seed)

# Dataset

In [ ]:
TRAIN_PATH = f'{BASICCONF.data_path}/train.csv'
TEST_PATH = f'{BASICCONF.data_path}/test.csv'
SAMP_SUB = f'{BASICCONF.data_path}/sample_submission.csv'

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
samp_sup = pd.read_csv(SAMP_SUB)

## split CV

In [ ]:
def split_cv(conf, df_):
    df = df_.copy(deep=True)
    mskf = MultilabelStratifiedKFold(n_splits=conf.num_folds, shuffle=True, random_state=conf.seed)
    y = df[conf.target_cols]

    for n, (train_index, valid_index) in enumerate(mskf.split(df, y)):
        df.loc[ｖalid_index, 'fold'] = int(n)
    
    df['fold'] = df['fold'].astype(int)
        
    return df
        
# train_df = train_df.drop(train_df[train_df['text_id'] == 'F69C85F4C3CA'].index).reset_index(drop=True)
df = split_cv(BASICCONF, train_df)
df.groupby('fold').size()

fold
0    978
1    977
2    978
3    978
dtype: int64

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
# all_text = df['full_text']
# def ft_array_dist(full_texts, tokenizer):
#     lengths = []
#     for text in full_texts.fillna('').values:
#         length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
#         lengths.append(length)
#     lengths = np.array(lengths)
#     return lengths

# all_text_l = ft_array_dist(all_text, tokenizer)
# plt.hist(all_text_l, 40)
# ft_fold = df[df['fold'] != 1]['full_text']
# l_fold = ft_array_dist(ft_fold, tokenizer)
# plt.hist(l_fold, 40)

In [ ]:
class FB3Dataset(Dataset):
    def __init__(self, conf, df, tokenizer, fold):
        self.labels = df[conf.target_cols].reset_index(drop=True)
        self.full_texts = df['full_text'].reset_index(drop=True)
        self.tokenizer = tokenizer
        if conf.dynamic_max_len:
            self.max_len = self._get_max_len()
        else:
            self.max_len = conf.static_max_len_list[fold]
        
    def __len__(self):
        return len(self.full_texts)
    
    def __getitem__(self, idx):
        token = self._get_token(idx)
        label = self._get_label(idx)
        
        return token, label
    
    def _get_label(self, idx):
        return torch.tensor(self.labels.loc[idx].values, dtype=torch.float)
    
    def _get_token(self, idx):
        tokenized = self.tokenizer(
                        self.full_texts.loc[idx],
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding="max_length",
                        truncation=True,
                        return_tensors=None,
                )
        return {k: torch.tensor(v, dtype=torch.long) for k, v in tokenized.items()} # stack tensor
    
    # get longest max_len
    # https://www.kaggle.com/code/yasufuminakama/fb3-deberta-v3-base-baseline-train/notebook
    def _get_max_len(self):
        lengths = []
        for text in self.full_texts.fillna('').values:
            length = len(self.tokenizer(text, add_special_tokens=False)['input_ids'])
            lengths.append(length)
        return max(lengths) + 2 # cls & sep

# Model

In [ ]:
# https://www.kaggle.com/code/maunish/clrp-pytorch-roberta-finetune/notebook
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

# https://www.kaggle.com/competitions/feedback-prize-english-language-learning/discussion/361678
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, in_dim),
            nn.LayerNorm(in_dim),
            nn.GELU(),
            nn.Linear(in_dim, 1),
        )

    def forward(self, x, mask):
        w = self.attention(x).float()
        w[mask==0]=float('-inf')
        w = torch.softmax(w,1)
        x = torch.sum(w * x, dim=1)
        return x    
    
# https://www.kaggle.com/code/rhtsingh/utilizing-transformer-representations-efficiently
class HiddenAttentionPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_fc):
        super().__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_fc = hiddendim_fc
        self.dropout = nn.Dropout(0.1)

        q_t = torch.normal(mean=0.0, std=0.02, size=(1, self.hidden_size))
        self.q = nn.Parameter(q_t).float()
        w_ht = torch.normal(mean=0.0, std=0.02, size=(self.hidden_size, self.hiddendim_fc))
        self.w_h = nn.Parameter(w_ht).float()

    def forward(self, all_hidden_states):
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers+1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out = self.attention(hidden_states)
        out = self.dropout(out)
        return out

    def attention(self, h):
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = F.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        return v

class ConcatPooling(nn.Module):
    def __init__(self, pooling_last=4):
        super().__init__()
        self.pooling_last = pooling_last
        
    def forward(self, all_hidden_states):
        concat_pooling = torch.cat(tuple(all_hidden_states[-l] for l in range(1, self.pooling_last + 1)), -1)
#         concat_pooling = concat_pooling.mean(dim=1) # average instead of select only one
        concat_pooling = concat_pooling[:, 0] # select the first one
        return concat_pooling

# https://www.kaggle.com/competitions/google-quest-challenge/discussion/129840
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_layers=12, init_std=0.02):
        super().__init__()
        self.num_layers = num_layers
        weights_init = torch.zeros(self.num_layers).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, all_hidden_states):
        all_layer_encoders = torch.stack(
            [self.dropout(layer) for layer in all_hidden_states[-self.num_layers:]], dim=0
        )
        averaged_layers = (torch.softmax(self.layer_weights, dim=0).unsqueeze(1).unsqueeze(1).unsqueeze(1) * all_layer_encoders).sum(0)
        return averaged_layers
        
class MeanPooling(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class CustomModel(nn.Module):
    def __init__(self, conf, fold_num, config_path=None):
        super().__init__()
        if not config_path:
            self.model_conf = AutoConfig.from_pretrained(conf.model_name, output_hidden_states=True)
            self.model_conf = self._set_dropout(self.model_conf)
            self.backbone = AutoModel.from_pretrained(conf.model_name, config=self.model_conf)
        else:
            self.model_conf = torch.load(config_path)
            self.backbone = AutoModel.from_config(self.model_conf)
        if conf.gradient_checkpointing:
            self.backbone.gradient_checkpointing_enable()
        
        if not config_path:
            for layer in self.backbone.encoder.layer[-conf.reinit_last_layers:]:
                for module in layer.modules():
                    self._init_weights(module)
                    
        self.pooling_strategy = conf.pooling_strategy_list[fold_num]
        if self.pooling_strategy == 'mean_pooling':
            self.pooler = MeanPooling()
            
        elif self.pooling_strategy == 'concat_pooling':
            self.pooler = ConcatPooling(conf.concat_pooling_last)
            
        elif self.pooling_strategy == 'attn_pooling': 
            self.attn_pooler = AttentionPooling(self.model_conf.hidden_size) 
            for attn_module in self.attn_pooler.modules():
                self._init_weights(attn_module)
            
        elif self.pooling_strategy == 'wlp_attn_pooling':
            self.wlp_pooler = WeightedLayerPooling(self.model_conf.num_hidden_layers, self.model_conf.initializer_range)
            self.attn_pooler = AttentionPooling(self.model_conf.hidden_size)
            for attn_module in self.attn_pooler.modules():
                self._init_weights(attn_module)
                
        elif self.pooling_strategy == 'concat_h_attn_mean_pooling':
            self.hattn_pooler = HiddenAttentionPooling(self.model_conf.num_hidden_layers, self.model_conf.hidden_size, self.model_conf.hidden_size)
            self.mean_pooler = MeanPooling()

        elif self.pooling_strategy == 'concat_attn_mean_pooling':
            self.attn_pooler = AttentionPooling(self.model_conf.hidden_size)
            for attn_module in self.attn_pooler.modules():
                self._init_weights(attn_module)
            self.mean_pooler = MeanPooling()
            
        else:
            raise Exception('Invalid pooling strategy')

        if self.pooling_strategy in ['mean_pooling', 'attn_pooling', 'wlp_attn_pooling']:
            hidden_size = self.model_conf.hidden_size
        elif self.pooling_strategy in ['concat_pooling']:
            hidden_size = self.model_conf.hidenn_size * conf.concat_pooling_last
        elif self.pooling_strategy in ['concat_h_attn_mean_pooling', 'concat_attn_mean_pooling']:
            hidden_size = self.model_conf.hidden_size * 2
        else:
            raise Exception('Cannot create fc layer.')
            
        self.multi_dropout = conf.multi_dropout
        if self.multi_dropout:
            self.dropout1 = nn.Dropout(conf.multi_dropout_p[0])
            self.dropout2 = nn.Dropout(conf.multi_dropout_p[1])
            self.dropout3 = nn.Dropout(conf.multi_dropout_p[2])
            self.dropout4 = nn.Dropout(conf.multi_dropout_p[3])
            self.dropout5 = nn.Dropout(conf.multi_dropout_p[4])
        else:
            self.dropout0 = nn.Dropout(0.1)
        
        self.fc = nn.Linear(hidden_size, conf.num_labels)
        if conf.reinit_method is not None:
            self._init_weights2_([self.fc], conf.reinit_method)
        else:
            self._init_weights(self.fc)

        self.use_ln = conf.use_ln
        if self.use_ln:
            self.ln = nn.LayerNorm(hidden_size)
            self._init_weights(self.ln)
        
    def _set_dropout(self, conf, ratio=0.):
        conf.attention_probs_dropout_prob = ratio
        conf.hidden_dropout = ratio 
        conf.hidden_dropout_prob = ratio
        conf.pooler_dropout = ratio
        return conf

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.model_conf.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.model_conf.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def _init_weights2_(self, module_lst, method):
        for module in module_lst:
            for param in module.parameters():
                if param.dim() > 1:
                    if method == 'kaiming_normal':
                        nn.init.kaiming_normal_(param)
                    elif method == 'xavier_normal':
                        nn.init.xavier_normal_(param)
                    elif method == 'orthoganol':
                        nn.init.orthogonal_(param)
                    else:
                        raise Exception('The method is invalid')
        
    def forward(self, inputs):
        backbone_outputs = self.backbone(**inputs)
        if self.pooling_strategy == 'mean_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            pooler_outputs = self.pooler(last_hidden_states, inputs['attention_mask'])
            
        elif self.pooling_strategy == 'concat_pooling':
            all_hidden_states = torch.stack(backbone_outputs['hidden_states'])
            pooler_outputs = self.pooler(all_hidden_states)
            
        elif self.pooling_strategy == 'attn_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            pooler_outputs = self.attn_pooler(last_hidden_states, inputs['attention_mask'])
            
        elif self.pooling_strategy == 'wlp_attn_pooling':
            all_hidden_states = torch.stack(backbone_outputs['hidden_states'])
            wlp_pooler = self.wlp_pooler(all_hidden_states)
            pooler_outputs = self.attn_pooler(wlp_pooler, inputs['attention_mask'])
            
        elif self.pooling_strategy == 'concat_h_attn_mean_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            all_hidden_states = torch.stack(backbone_outputs['hidden_states'])
            hattn_outputs = self.hattn_pooler(all_hidden_states)
            mean_outputs = self.mean_pooler(last_hidden_states, inputs['attention_mask'])
            pooler_outputs = torch.cat((hattn_outputs, mean_outputs), -1)
            
        elif self.pooling_strategy == 'concat_attn_mean_pooling':
            last_hidden_states = backbone_outputs['last_hidden_state']
            attn_outputs = self.attn_pooler(last_hidden_states, inputs['attention_mask'])
            mean_outputs = self.mean_pooler(last_hidden_states, inputs['attention_mask'])
            pooler_outputs = torch.cat((attn_outputs, mean_outputs), -1)
            
        if self.use_ln:
            pooler_outputs = self.ln(pooler_outputs)
            
        if self.multi_dropout:
            x1 = self.fc(self.dropout1(pooler_outputs))
            x2 = self.fc(self.dropout2(pooler_outputs))
            x3 = self.fc(self.dropout3(pooler_outputs))
            x4 = self.fc(self.dropout4(pooler_outputs))
            x5 = self.fc(self.dropout5(pooler_outputs))
            
            outputs = (x1 + x2 + x3 + x4 + x5) / 5

        else:
            outputs = self.fc(self.dropout0(pooler_outputs))
            
        return outputs

In [ ]:
# https://www.kaggle.com/code/wht1996/feedback-nn-train/notebook
class AWP:
    def __init__(
        self,
        model,
        criterion,
        optimizer,
        adv_param="weight",
        adv_lr=1,
        adv_eps=0.2,
        adv_step=1,
        scaler=None,
        apex=False,
    ):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.adv_step = adv_step
        self.backup = {}
        self.backup_eps = {}
        self.scaler = scaler
        self.apex = apex

    def attack_backward(self, inputs, labels):
        self._save() 
        for i in range(self.adv_step):
            self._attack_step() 
            with torch.cuda.amp.autocast(enabled=self.apex):
                adv_outputs = self.model(inputs)
                adv_loss = self.criterion(adv_outputs, labels)
                adv_loss = adv_loss.mean()
            self.optimizer.zero_grad()
            self.scaler.scale(adv_loss).backward()
            
        self._restore()

    def _attack_step(self):
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]), self.backup_eps[name][1]
                    )
                # param.data.clamp_(*self.backup_eps[name])

    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _restore(self,):
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

# Loss Function

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.__name__ = self.__class__.__name__
    
    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Helper

In [ ]:
# https://realpython.com/python-timer/
class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self.split_time = []
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
            
        self._start_time = None
    
    def get_time(self):
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
            
        return time.perf_counter() - self._start_time
    
    def split(self):
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
            
        self.split_time.append(time.perf_counter() - self._start_time)
    
    def get_split_time(self, idx):
        return self.split_time[idx]
    
    @staticmethod
    def formatting(second):
        return str(datetime.timedelta(seconds=round(second)))

In [ ]:
# https://www.kaggle.com/code/yasufuminakama/fb3-deberta-v3-base-baseline-train/notebook
class Averager:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
    def get_average(self):
        return self.avg
    
    def get_value(self):
        return self.val

In [ ]:
class ModelWeightAverager:
    def __init__(self, conf, fold):
        self.conf = conf
        self.fold = fold
        self.state_dict_list = []
    
    def add_state_dict(self, state_dict):
        self.state_dict_list.append(state_dict)
        
    def average_state_dict(self):
        master_sd = CustomModel(self.conf, self.fold).state_dict()
        for key in master_sd:
            master_sd[key] = 0
            for state_dict in self.state_dict_list:
                master_sd[key] += state_dict[key]
            master_sd[key] = master_sd[key] / len(self.state_dict_list)
        
        torch.save({
            'model_state_dict': master_sd,
        },
            Path(self.conf.save_path, f'best-epoch-fold{self.fold}-swa.pt'))
        
        print(f'SAVED FOLD{self.fold}_M_SWA)')

In [ ]:
# optimize padding size
# https://www.kaggle.com/code/yasufuminakama/fb3-deberta-v3-base-baseline-train/notebook
def collator(inputs):
    mask_len = int(inputs['attention_mask'].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:, :mask_len]
    return inputs

def get_dataloader(conf, df, tokenizer, fold_num):
    
    train_dataset = FB3Dataset(conf, df[df['fold'] != fold_num], tokenizer, fold_num)
    valid_dataset = FB3Dataset(conf, df[df['fold'] == fold_num], tokenizer, fold_num)
    
    total_train_samples = len(train_dataset)
    
#     data_collator = DataCollatorWithPadding(tokenizer, padding='longest', return_tensors=None)
    
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=conf.num_batch,
#         collate_fn=data_collator,
        num_workers=4,
        shuffle=True,
        pin_memory=True,
        drop_last=True
    )
    
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=conf.num_batch,
#         collate_fn=data_collator,
        num_workers=4,
        shuffle=False,
        pin_memory=True,
        drop_last=False
    )
    
    return train_dataloader, valid_dataloader, total_train_samples

def get_model(conf, fold_num):
    model = CustomModel(conf, fold_num)
    model_config_file = Path(conf.save_path, Path(conf.model_name).name + '_config.pt')
    
    # save model config file
    if not model_config_file.is_file():
        torch.save(model.model_conf, model_config_file)
        
    return model

def get_tokenizer(conf):
    tokenizer = AutoTokenizer.from_pretrained(conf.model_name)
    tokenizer_file = Path(conf.save_path, 'tokenizers/')
    
    if not tokenizer_file.is_file():
        tokenizer.save_pretrained(tokenizer_file) # save tokenizer vocab
        
    return tokenizer

def get_optimizer(conf):
    optimizer_dict = {
        'adamw': optim.AdamW,
    }
    return optimizer_dict[conf.optimizer]

def get_optimizer_grouped_params(model, bb_lr, ll_lr, weight_decay=0.01, layerwise_learning_rate_decay=0.9):
    # turn off weight decay in some layer
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if 'backbone' not in n],
            "weight_decay": 0.0,
            "lr": ll_lr,
        },
    ]
    # layer-wise learning rate decay
    layers = [model.backbone.embeddings] + list(model.backbone.encoder.layer)
    layers.reverse()
    decay_lr = bb_lr
    for layer in layers:
        decay_lr *= layerwise_learning_rate_decay
        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": decay_lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": decay_lr,
            },
        ]
    return optimizer_grouped_parameters

def get_scheduler(conf, total_samples):
    scheduler_dict = {
        'cosine_warmup': {
            'scheduler': transformers.get_cosine_schedule_with_warmup,
            'params': {
                'num_warmup_steps': int((total_samples // conf.num_batch) * conf.warmup_epoch),
                'num_training_steps': (total_samples // conf.num_batch) * conf.num_epochs,
            }
        },
        'linear_warmup': {
            'scheduler': transformers.get_linear_schedule_with_warmup,
            'params': {
                'num_warmup_steps': int((total_samples // conf.num_batch) * conf.warmup_epoch),
                'num_training_steps': (total_samples // conf.num_batch) * conf.num_epochs,
            }
        },
        'cosine_restart_warmup': {
            'scheduler': transformers.get_cosine_with_hard_restarts_schedule_with_warmup,
            'params': {
                'num_warmup_steps':  int((total_samples // conf.num_batch) * conf.warmup_epoch),
                'num_training_steps': (total_samples // conf.num_batch) * conf.num_epochs,
                'num_cycles': 2,
            }
        },
    }
    
    return scheduler_dict[conf.scheduler]['scheduler'], scheduler_dict[conf.scheduler]['params']

def scores_with_MCRMSE(y_predicteds, y_targets):
    cols_score = []
    col_wise = y_targets.shape[1]
    for col in range(col_wise):
        y_predicted = torch.as_tensor(y_predicteds[:, col])
        y_target = torch.as_tensor(y_targets[:, col])
        rmse = mean_squared_error(y_predicted, y_target, squared=False) # returns RMSE value if set to False
        cols_score.append(rmse)
    score_mean = torch.mean(torch.as_tensor(cols_score))
    return score_mean, cols_score

def calculate_fold_mean(conf, df):
    bsr_list = []
    for i in range(conf.num_folds):
        fold = df[df['fold'] == i]
        best_score_row = fold.loc[fold['mcrmse'] == fold['mcrmse'].min()]
        bsr_list.append(best_score_row)
        
    cv_df = pd.concat(bsr_list)
    cv_mean = cv_df.mean(axis=0)
    cv_mean['fold'] = cv_mean['epoch'] = int(99)
    cv_mean = cv_mean.to_frame().T
    cv_df = pd.concat([cv_df, cv_mean], axis=0)
    return cv_df.reset_index(drop=True)

def get_oof_df(target_cols, predictions, targets):
    oof_df = pd.DataFrame()
    oof_df[target_cols] = targets.cpu().numpy()
    oof_df[[f'pred_{col}' for col in target_cols]] = predictions.cpu().numpy()
    return oof_df

def calculate_oof_cv(target_cols, oof_df):
    predictions = oof_df[[f'pred_{col}' for col in target_cols]].values
    targets = oof_df[target_cols].values
    score_mean, cols_score = scores_with_MCRMSE(predictions, targets)
    return score_mean, cols_score

def export_config(basic, train):
    config_dict = {**basic.__dict__, **train.__dict__}
    remove_keys_list = ['__module__', '__dict__', '__weakref__', '__doc__']
    
    for key in remove_keys_list:
        config_dict.pop(key)
    
    with open(Path(train.save_path, 'config.yml'), 'w') as file:
        yaml.dump(config_dict, file)

# Trainer

In [ ]:
class Trainer:
    def __init__(self, conf, device, fold, model, optimizer, optimizer_grouped_params, scheduler, scheduler_params):
        self.device = device
        self.current_fold = fold
        self.model = model
        self.optimizer = optimizer(optimizer_grouped_params, lr=conf.ll_lr, eps=conf.optim_eps)
        self.scheduler = scheduler(self.optimizer, **scheduler_params)
        
        self.dry_run = conf.dry_run
        self.exp = conf.exp

        self.model_name = conf.model_name
        self.target_cols = conf.target_cols
        self.use_apex = conf.use_apex
        self.use_awp = conf.use_awp
        self.use_swa = conf.use_swa
        self.multi_dropout = conf.multi_dropout
        self.grad_clip = conf.grad_clip
        self.grad_max_norm = conf.grad_max_norm
        
        self.num_batch = conf.num_batch
        self.num_epochs = conf.num_epochs
        self.current_epoch = None
        self.batch_scheduler = conf.batch_scheduler
        self.verbose_step = conf.verbose_step
        self.accumulate_step = conf.accumulate_step

        self.save_path = conf.save_path
        
        self.scaler = GradScaler(enabled=self.use_apex)
        self.criterion = nn.SmoothL1Loss(beta=conf.loss_beta) # [RMSELoss, SmoothL1Loss]
        
        self.awp_start_epoch = conf.awp_start_epoch
        if self.use_awp:
            self.awp = AWP(
                model=self.model,
                criterion=self.criterion,
                optimizer=self.optimizer,
                adv_lr=conf.adv_lr,
                adv_eps=conf.adv_eps,
                scaler=self.scaler,
                apex=self.use_apex,
            )
            
        self.swa_start_step_ratio = conf.swa_start_step_ratio
        if self.use_swa:
            self.swa_model = AveragedModel(self.model)
            self.swa_scheduler = SWALR(
                self.optimizer,
                swa_lr=conf.swa_lr,
                anneal_strategy=conf.swa_anneal_strat,
            )

        self.best_train_loss = torch.tensor(10000) # placeholder
        self.best_valid_loss = torch.tensor(10000) # placeholder
        self.best_score = torch.tensor(10000) # placeholder
        self.best_score_col = [torch.tensor(10000) for i in range(6)] # placeholder
        
        self.record_cols = [
            'fold',
            'epoch',
            'train_loss',
            'valid_loss',
            'mcrmse',
            'cohesion',
            'syntax',
            'vocabulary',
            'phraseology',
            'grammar',
            'conventions',
        ]
        self.record_df = pd.DataFrame(columns=self.record_cols)
        
        if self.dry_run: 
            self.log('=============')
            self.log('== DRY_RUN ==')
            self.log('=============')
            

        mode = 'EXPERIMENTING_MODE' if self.exp else 'CV_MODE'
        current_max_len = 'dynamic_padding' if conf.dynamic_max_len else conf.static_max_len_list[self.current_fold]
        current_pooling_strategy = conf.pooling_strategy_list[self.current_fold]
        
        self.log(f'Date: {datetime.datetime.now(pytz.timezone("Asia/Ho_Chi_Minh") )} (GMT+7)')
        self.log(f'Mode: {mode}')
        self.log(f'Train_on: {self.device}, (AMP: {self.use_apex}, GradScaler: {self.scaler.is_enabled()})')
        self.log(f'Model: {self.model_name}')
        self.log(f'Model_config: {self.model.model_conf}')
        self.log(f'Pooling_strategy: {current_pooling_strategy}')
        self.log(f'Initailzation: {conf.reinit_method}')
        self.log(f'AWP: {self.use_awp} (adv_lr: {conf.adv_lr}, adv_eps: {conf.adv_eps}) at epoch {self.awp_start_epoch}')
        self.log(f'SWA: {self.use_swa} (swa_lr: {conf.swa_lr}, anneal_strat: {conf.swa_anneal_strat}) at last {conf.swa_start_step_ratio}')
        self.log(f'Multi_sample_dropout: {self.multi_dropout} (p: {conf.multi_dropout_p})')
        self.log(f'Loss_fn: {str(self.criterion)}')
        self.log(f'Optimizer: {optimizer.__name__}')
        self.log(f'LR: (Backbone: {conf.bb_lr}, LowerLayer: {conf.ll_lr})')
        self.log(f'LR_Scheduler: {scheduler.__name__} {scheduler_params}')
        self.log(f'Grad_clip_norm: {self.grad_clip} (max_norm: {self.grad_max_norm})')
        self.log(f'Number_of_batches: {self.num_batch} (Gradient_accumulate: {self.accumulate_step})')
        self.log(f'max_len: {current_max_len}')
        self.log('')
        
    def fit(self, train_loader, valid_loader):
        for epoch in range(self.num_epochs):
            self.current_epoch = epoch + 1
            
            timer = Timer()
            timer.start()
            
            self.log('TRAIN_LOOP')
            train_loss = self._train_fn(train_loader)
            self.log('')
            
            self.log('VALID_LOOP')
            valid_loss, score_mean, cols_score, valid_predictions, valid_targets = self._valid_fn(valid_loader)
            self.log('')
            
            epoch_summary_dict = {
                'EPOCH': [f'{self.current_epoch}/{self.num_epochs}'],
                'TRAIN_LOSS': [f'{train_loss:.5f}'],
                'VALID_LOSS': [f'{valid_loss:.5f}'],
                'MCRMSE': [f'{score_mean:.5f}'],
                'COLS': [' | '.join([f'{col:.3f}' for col in cols_score])],
                'TIME': [Timer.formatting(timer.get_time())],
            }
            epoch_summary_table = tabulate(epoch_summary_dict, headers='keys', tablefmt="github")
            
            
            self.log('--------------------')
            self.log(f'EPOCH: {self.current_epoch}/{self.num_epochs} SUMMARY')
            self.log('--------------------')
            self.log(epoch_summary_table)
            self.log('')
            
            timer.stop()
            
            torch.cuda.empty_cache()
            gc.collect()

            self._compare_and_save(score_mean, cols_score, train_loss, valid_loss, valid_predictions, valid_targets)
                
            self.record([
                self.current_fold,
                self.current_epoch,
                train_loss.detach().cpu().numpy(),
                valid_loss.detach().cpu().numpy(),
                score_mean.cpu().numpy(),
                *[col.cpu().numpy() for col in cols_score],
            ])
            
            if self.dry_run: break
            # end of the loop
        
        if self.use_swa:
            torch.optim.swa_utils.update_bn(train_loader, self.swa_model)

        fold_summary_dict = {
            'MCRMSE': [f'{self.best_score:.5f}'],
            'cohesion': [f'{self.best_score_col[0]:.5f}'],
            'syntax': [f'{self.best_score_col[1]:.5f}'],
            'vocabulary': [f'{self.best_score_col[2]:.5f}'],
            'phraseology': [f'{self.best_score_col[3]:.5f}'],
            'grammar': [f'{self.best_score_col[4]:.5f}'],
            'conventions': [f'{self.best_score_col[5]:.5f}'],
        }
        
        fold_summary_table = tabulate(fold_summary_dict, headers='keys', tablefmt='simple_grid')
        
        self.log('')
        self.log(('-' * 35) + f' FOLD {self.current_fold} RESULT ' + ('-' * 35))
        self.log(fold_summary_table)
        self.log('')
        self.log(('#' * 35) + f' END OF FOlD {self.current_fold} ' + ('#' * 35))
        self.log('')
        self.log('')
        
        return self.record_df, self.oof_df
        
    def _train_fn(self, train_loader):
        self.model.train()
        total_loss = Averager()
        current_lr = self.scheduler.get_lr()[0] # placeholder
        timer = Timer()
        timer.start()
        if self.use_awp and self.awp_start_epoch <= self.current_epoch: self.log('AWP_ACTIVATED')
        
        for step, (inputs, labels) in enumerate(train_loader):

            inputs = collator(inputs)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            labels = labels.to(self.device)
            batchsize = len(labels)
            
            with autocast(enabled=self.use_apex):
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

            if self.accumulate_step > 1:
                loss = loss / self.accumulate_step
            total_loss.update(loss, batchsize)
            
            self.scaler.scale(loss).backward()
            
            if (step + 1) % self.accumulate_step == 0:
                if self.use_awp and self.awp_start_epoch <= self.current_epoch:
                    self.awp.attack_backward(inputs, labels)
                
                if self.use_swa:
                    swa_start_step = int((self.num_epochs * len(train_loader)) * (1 - self.swa_start_step_ratio))
                    current_training_step = int(((self.current_epoch - 1) * len(train_loader)) + step)
                    if current_training_step == swa_start_step:
                        self.log('SWA_ACTIVATED')
                    if current_training_step >= swa_start_step:
                        self.swa_model.update_parameters(self.model)
                        self.swa_scheduler.step()
                        current_lr = self.swa_scheduler.get_last_lr()[0]
                    else:
                        if self.batch_scheduler:
                            self.scheduler.step()
                            current_lr = self.scheduler.get_lr()[0]
                else:
                    if self.batch_scheduler:
                        self.scheduler.step()
                        current_lr = self.scheduler.get_lr()[0]
                    
                if self.grad_clip:
                    self.scaler.unscale_(self.optimizer)
                    grad_norm = torch.nn.utils.clip_grad_norm_(
                        parameters=self.model.parameters(),
                        max_norm=self.grad_max_norm,
                    )

                self.scaler.step(self.optimizer)
                self.scaler.update()
                self.optimizer.zero_grad()

            if step % self.verbose_step == 0 or step == (len(train_loader) - 1):
                self.log(
                    f'[TRAIN F{self.current_fold}] ' + \
                    f'EPOCH: {self.current_epoch}/{self.num_epochs} | ' + \
                    f'STEP: {str(step).zfill(len(str(len(train_loader))))}/{len(train_loader)} | ' + \
                    f'LOSS: {total_loss.get_value():.5f} ({total_loss.get_average():.5f}) | ' + \
                    f'LR: {current_lr:.8f} | ' + \
                    f'TIME: {Timer.formatting(timer.get_time())} |'
                )

            if self.dry_run: break
            # end of the loop
            
        timer.stop()

        return total_loss.get_average()
            
    
    def _valid_fn(self, valid_loader):
        self.model.eval()
        total_loss = Averager()
        timer = Timer()
        timer.start()
        outputs_list = [] # for stacking outputs
        targets_list = [] # for stacking labels
        
        for step, (inputs, labels) in enumerate(valid_loader):
            
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            labels = labels.to(self.device)
            batchsize = len(labels)
            
            with torch.no_grad():
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
            total_loss.update(loss, batchsize)
            outputs_list.append(outputs)
            targets_list.append(labels)
            
            if step % self.verbose_step == 0 or step == (len(valid_loader) - 1):
                self.log(
                    f'[VALID F{self.current_fold}] ' + \
                    f'EPOCH: {self.current_epoch}/{self.num_epochs} | ' + \
                    f'STEP: {str(step).zfill(len(str(len(valid_loader))))}/{len(valid_loader)} | ' + \
                    f'LOSS: {total_loss.get_value():.5f} ({total_loss.get_average():.5f}) | ' + \
                    f'TIME: {Timer.formatting(timer.get_time())} |'
                )

            if self.dry_run: break
            # end of the loop

        predictions = torch.cat(outputs_list)
        targets = torch.cat(targets_list)

        score_mean, cols_score = scores_with_MCRMSE(predictions, targets)
        
        timer.stop()
        
        return total_loss.get_average(), score_mean, cols_score, predictions, targets
    
    def _compare_and_save(self, score, col_score, train_loss, valid_loss, valid_predictions, valid_targets):
        if score < self.best_score:
            self.best_score = score
            self.best_score_col = col_score
            self.best_train_loss = train_loss
            self.best_valid_loss = valid_loss
            
            file_name = f'best-epoch-fold{self.current_fold}.pt'
            self._save(self.save_path, file_name, last_checkpoint=False)
            
            self.oof_df = get_oof_df(self.target_cols, valid_predictions, valid_targets)
        
            self.log('')
            self.log(f'[SAVED] EPOCH: {self.current_epoch} | MCRMSE: {self.best_score}')
            self.log('')
    
    def _save(self, save_path, file_name, last_checkpoint=False):
        self.model.eval()
        if last_checkpoint:
            torch.save({
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scaler': self.scaler.state_dict(),
                'best_train_loss': self.best_train_loss,
                'best_valid_loss': self.best_valid_loss,
                'best_score': self.best_score,
                'epoch': self.current_epoch,
            }, Path(save_path, file_name))
        else:
            torch.save({
                'model_state_dict': self.model.state_dict(),
                'best_train_loss': self.best_train_loss,
                'best_valid_loss': self.best_valid_loss,
                'best_score': self.best_score,
                'epoch': self.current_epoch,
            }, Path(save_path, file_name))
            
    def log(self, message):
        print(message)
        if not self.dry_run:
            with open(Path(self.save_path, 'log.txt'), mode='a+', encoding='utf-8') as log:
                log.write(f'{message}\n')

    def record(self, record_row):
        new_record_dict = {k: [v] for k, v in zip(self.record_cols, record_row)}
        new_record = pd.DataFrame.from_dict(new_record_dict)
        self.record_df = pd.concat([self.record_df, new_record], axis=0)

In [ ]:
def run_training(train_df, conf, dry_run=True, exp=True):
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
    conf.dry_run = dry_run # for debugging
    conf.exp = exp # for experimenting train only the first fold
    
    tokenizer = get_tokenizer(conf)
    train_result_df = pd.DataFrame()
    oof_df = pd.DataFrame()
    
#     for fold in range(conf.num_folds):
    for fold in conf.train_fold_list:
        
        seed_everything(conf.seed)
        model = get_model(conf, fold)
        model = model.to(device)
        optimizer = get_optimizer(conf)
        optimizer_grouped_params = get_optimizer_grouped_params(model, conf.bb_lr, conf.ll_lr, conf.weight_decay, conf.llrd)
        train_loader, valid_loader, total_train_samples = get_dataloader(conf, train_df, tokenizer, fold)
        scheduler, scheduler_params = get_scheduler(conf, total_train_samples)

        trainer = Trainer(conf, device, fold, model, optimizer, optimizer_grouped_params, scheduler, scheduler_params)
        fold_result, fold_oof_df = trainer.fit(train_loader, valid_loader)
        train_result_df = pd.concat([train_result_df, fold_result], axis=0)
        oof_df = pd.concat((oof_df, fold_oof_df), axis=0)
        
        if conf.dry_run or conf.exp : break
    
    cv_result_df = calculate_fold_mean(conf, train_result_df)
    oof_score_mean, oof_cols_score = calculate_oof_cv(conf.target_cols, oof_df)
    cv_df = pd.DataFrame([[oof_score_mean.numpy(), *[col.numpy() for col in oof_cols_score]]], columns=['mcrmse', *conf.target_cols])
    
    print('Overall Training Result')
    display(train_result_df)
    print('Best Result')
    display(cv_result_df)
    print('CV Result')
    display(cv_df)
    
    train_result_df.reset_index(drop=True).to_csv(Path(conf.save_path, 'train_result.csv'), index=False)
    cv_result_df.reset_index(drop=True).to_csv(Path(conf.save_path, 'best_result.csv'), index=False)
    cv_df.reset_index(drop=True).to_csv(Path(conf.save_path, 'cv_result.csv'), index=False)

In [ ]:
class TRAINCONF(BASICCONF):
    
    # general config
    exp_num = '0'
    save_path = f'/content/drive/MyDrive/Colab Notebooks/kaggle/fb3/results/exp{exp_num}'
    
     # run config
    dry_run = True # for debugging
    exp = True # for experimenting
    train_fold_list = [0, 1, 2, 3]
    
    # dataset config
    dynamic_max_len = False
    static_max_len_list = [768, 768, 768, 768] # if dynaimic_padding is False: then use static_max_len_list instead
    # [1428, 1428, 1428, 1428] default max_len for 4 fold
    
    # model config
    model_name = 'microsoft/deberta-v3-base' # [xsmall, small, base, large, xlarge]
    reinit_last_layers = 1 # from the bottom (no need to set negative value).
    reinit_method = None # [kaiming_normal, xavier_normal, orthoganol, None] # if None: initialize by  normal dist
    gradient_checkpointing = True
    
    use_awp = True
    awp_start_epoch = 1
    adv_lr = 2e-5
    adv_eps = 1e-3
    
    use_swa = False
    swa_start_step_ratio = 0.112 #tart from last n% of the training step # 0.112(1300)
    swa_lr = 1e-6
    swa_anneal_strat = 'cos' # ['cos', 'linear']
    
    pooling_strategy_list = [
        'concat_attn_mean_pooling',
        'concat_attn_mean_pooling',
        'concat_attn_mean_pooling',
        'concat_attn_mean_pooling',
    ]
    
    # [
    #     mean_pooling,
    #     concat_pooling,
    #     attn_pooling,
    #     wlp_attn_pooling,
    #     concat_h_attn_mean_pooling,
    #     concat_attn_mean_pooling,
    # ]
    
    wlp_pooling_start = 0 # for weighted layer pooling (n from the top)
    concat_pooling_last = 4 # for concatenate pooling (n from the bottom)
    
    use_ln = False

    multi_dropout = True
    multi_dropout_p = [0.3, 0.3, 0.3, 0.3, 0.3]
    
    # trainer config
    use_apex = torch.cuda.is_available()
    verbose_step = 40
    num_epochs = 4
    num_batch = 8
    accumulate_step = 1
    
    # optimizer config
    optimizer = 'adamw'
    bb_lr = 2e-5 # backbone lr
    ll_lr = 3e-5 # lowerlayer lr
    
    weight_decay = 0.01
    llrd = 0.9
    optim_eps = 1e-6
    
    batch_scheduler = True
    scheduler = 'cosine_warmup' # [cosine_warmup, linear_warmup, consine_restart_warmup]
    warmup_epoch = 0.25 # first n of the first epoch
    
    grad_clip = False
    grad_max_norm = 10
    
    # loss_fn config
    loss_beta = 1.0

In [ ]:
def check_or_create(path):
    path = Path(path)
    if not path.is_dir():
        path.mkdir(parents=True, exist_ok=True)

# RUN TRAINING

In [ ]:
check_or_create(TRAINCONF.save_path)
export_config(basic=BASICCONF, train=TRAINCONF)
run_training(df, TRAINCONF, dry_run=False, exp=False)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


Date: 2022-11-16 20:17:07.566016+07:00 (GMT+7)
Mode: CV_MODE
Train_on: cuda, (AMP: True, GradScaler: True)
Model: microsoft/deberta-base
Model_config: DebertaConfig {
  "_name_or_path": "microsoft/deberta-base",
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0.0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "c2p",
    "p2c"
  ],
  "position_biased_input": false,
  "relative_attention": true,
  "transformers_version": "4.20.1",
  "type_vocab_size": 0,
  "vocab_size": 50265
}

Pooling_strategy: concat_attn_mean_pooling
Initailzation: None
AWP: True (adv_lr:

RuntimeError: ignored

# Jump to Results 🔼

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(TRAINCONF.model_name)
# train_loader, valid_loader = get_dataloader(df, tokenizer, TRAINCONF.num_batch, 0)
# loader = iter(train_loader)

# batch = next(loader)

In [ ]:
# !rm -r '/content/drive/MyDrive/Colab Notebooks/kaggle/fb3/results/exp2/'

In [ ]:
# 0.4493041, tl-0.09330081, vl-0.100785285